# Resolution and Contrast Evaluation

This notebook provides the code for the resolution evaluation on a PICMUS real world data-set. Furthermore, there is the contrast evaluation of a circular echoic region.

The notebook is layed out in a way that two beamformers are applied to the same data and evaluated using the same metrics. It is recommended to leave the first beamformer as a DAS beamformer to have a baseline and use the beamformer class one wants to scrutinize as the second one.

In [ ]:
# Import PyBF library
import numpy as np
import sys, os
import h5py
'''
import sys, os
look here if below import doesn't work: https://stackoverflow.com/questions/7505988/importing-from-a-relative-path-in-python
'''
# sys.path.append('../jupyter')
sys.path.append('.')
from pybf.pybf.io_interfaces import DataLoader
from pybf.scripts.beamformer_DAS_ref import BFCartesianReference
from pybf.scripts.beamformer_mvbf_spatial_smooth import BFMVBFspatial
from pybf.scripts.beamformer_mvbf_DCR import BFMVBFdcr
from pybf.pybf.image_settings import ImageSettings
from pybf.pybf.visualization import plot_image
from pybf.scripts.picmus_eval import PicmusEval

%matplotlib ipympl

### Import data

In [ ]:
data_loader_obj = DataLoader('./pybf/tests/data/Picmus/resolution_distorsion/rf_dataset.hdf5')

In [ ]:
tx_temp = data_loader_obj.tx_strategy
tx_temp

 ### Image settings
  
For some beamformers evaluated in the project, the tx_strategy needs to be set up in a way that at least two plane waves need to be emitted from different angles.

 ## Instantiate beamformers

(Copied from the PyBF code comments)

### `active_elements`
- sets the active elements of the transducer by list of indices
- Attention: elements numeration starts from 0

### `channel_reduction`

- channel reduction is the number of channels to be used for beamforming

Code snipped from `beamformer_DAS_ref.py`

```python
ch_nr = n_elements
start_i = int(np.ceil((ch_nr - channel_reduction)/2))
stop_i = int(start_i + channel_reduction)
```

 ## Beamform
 
 Performs the beamforming for the whole image
 
 **Note:** This part of the code is by far the most computationally expensive step. Beamforming using a MVBF beamformer can take several minutes (up to approx. 20 minutes) if a high number of channels is used!

## Evaluations and Visualize

For both beamformers the Full-Width at Half-Maximum (FWHM) metric is calculated for a number of predefined scatterers in the test data in meters.

In [ ]:
# all imports


Defining all functions

In [ ]:
def create_beamformer(tx_strategy_indices, img_config):

    ### Specify preprocessing parameters for RF data ###
    decimation_factor = 1
    interpolation_factor = 10

    ### Specify TX strategy and Apodization parameters ###
    start_time = 0
    correction_time_shift = 0

    alpha_fov_apod = 40

    tx_strategy = [[],[]]
    # take tx_strategy and outputs a list with all the angles we want to use
    # data_loader.tx_strategy[1] are the angles
    # our tx_strategy_indices is a list of 2 elements: the first is the name of the strategy, the second is the list of angles
        # tx_strategy_indices = ['PW_name', [30, 40, 50, 60]]
    # we want to return an list with ['PW_name', [data_loader_obj.tx_strategy[1][30], ...]]
    all_angles = data_loader_obj.tx_strategy[1]
    # slice all_angles with indices in tx_strategy_indices to get the correct angles
    tx_strategy[0] = tx_strategy_indices[0]
    tx_strategy[1] = all_angles[tx_strategy_indices[1]]
    print(tx_strategy)

    # tx_strategy = ['PW_4_2.2', [data_loader_obj.tx_strategy[1][angle_indices[0]], data_loader_obj.tx_strategy[1][angle_indices[1]], data_loader_obj.tx_strategy[1][angle_indices[2]], data_loader_obj.tx_strategy[1][angle_indices[3]] ]]
    # tx_strategy = ['PW_75_16', data_loader_obj.tx_strategy[1]]
    # tx_strategy = ['PW_4_2.1', [data_loader_obj.tx_strategy[1][33], data_loader_obj.tx_strategy[1][37], data_loader_obj.tx_strategy[1][38], data_loader_obj.tx_strategy[1][42]]]
    # tx_strategy = ['PW_2_2', [data_loader_obj.tx_strategy[1][0], data_loader_obj.tx_strategy[1][1]]]
    rf_data_shape = (len(tx_strategy[1]),) + data_loader_obj.get_rf_data(0, 0).shape
    rf_data = np.zeros(rf_data_shape)
    inclin_index = np.asarray(tx_strategy_indices[1])
    for i in range(rf_data.shape[0]):
        rf_data[i, :, :] = data_loader_obj.get_rf_data(0, inclin_index[i])

    SAMPLING_FREQ = data_loader_obj.f_sampling

    filters_params = [1 * 10 **6, 8 * 10 **6, 0.5 * 10 **6]

    bf = BFCartesianReference(SAMPLING_FREQ,
                                tx_strategy,
                                data_loader_obj.transducer,
                                decimation_factor,
                                interpolation_factor,
                                img_res,
                                img_config,
                                start_time=start_time,
                                correction_time_shift=correction_time_shift,
                                alpha_fov_apod=alpha_fov_apod,
                                bp_filter_params=filters_params,
                                envelope_detector='I_Q',
                                picmus_dataset=True,
                                channel_reduction=128)
    
    return bf, rf_data

In [ ]:
def perform_beamforming(tx_strategy, transducer_indices):
    bf, rf_data = create_beamformer(tx_strategy)

    # set active transducer elements
    # indices = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120]
    sliced_rf_data = rf_data[:, transducer_indices, :]

    img_data = bf.beamform(sliced_rf_data, numba_active=False)

    return img_data

Immuntable Variables

In [ ]:
# immunatble variables

# below code generates list containting names of all versions of the images
# under each plane wave angle (folder) will be the range of different transducer counts

every_nth = ['2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th', '10th']
percent_from = [10, 20, 30, 40, 50, 60, 70, 80, 90]

# Trials using different count of transducers
transducer_trials_names = ['full']

# For loop to concatenate each string with the suffix
for string in every_nth:
    concatenated_string = 'every ' + string
    transducer_trials_names.append(concatenated_string)

for percent in percent_from:
    concatenated_string = str(percent) + '% from middle'
    transducer_trials_names.append(concatenated_string)

# Print the concatenated strings
print(transducer_trials_names)
print(len(transducer_trials_names))

# Trials using different amounts of plane wave angles
angle_trials_names = []

for percent in percent_from:
    concatenated_string = str(percent) + '% of angles'
    angle_trials_names.append(concatenated_string)

angle_trials_names.append('100% of plane wave angles')

# Print the concatenated strings
print(angle_trials_names)
print(len(angle_trials_names))

############################################################################
# generate numerical vectors that go with the names
# must be lists

# full
transducer_trials_values = [[i for i in range(128)]]
# every nth
for i in range(2, 11):
    transducer_trials_values.append([j for j in range(0, 128, i)])
# percent from middle
for i in range(90, 0, -10):
    ch_nr = round(128 * (i / 100.0))
    start_i = int(np.ceil((128 - ch_nr)/2))
    stop_i = int(start_i + ch_nr)
    transducer_trials_values.append([j for j in range(start_i, stop_i)])

# ch_nr = round(128 * (i / 10.0))
# start_i = int(np.ceil((ch_nr - channel_reduction)/2))
# stop_i = int(start_i + channel_reduction)

print(transducer_trials_values)
print(len(transducer_trials_values))

index = 6
print(transducer_trials_names[index])
print(transducer_trials_values[index])

angle_trials_values = []

# percent from middle
for i in range(10, 101, 10):
    ch_nr = round(75 * (i / 100.0))
    start_i = int(np.ceil((75 - ch_nr)/2))
    stop_i = int(start_i + ch_nr)
    angle_trials_values.append([j for j in range(start_i, stop_i)])

print(angle_trials_values)
print(len(angle_trials_values))

For Loops that generate all the data

In [ ]:
# temprary shorter list for testing
transducer_trials_names = ['full']
transducer_trials_values = [[j for j in range(0, 128)]]
print(transducer_trials_values)

# angle_trials_names = ['10% of plane wave angles']
# angle_trials_values = [angle_trials_values[-1]]
# print(angle_trials_values)

In [ ]:
# for loops
for j, angle_trial_name in enumerate(angle_trials_names):
    for i, transducer_trial_name in enumerate(transducer_trials_names):
        # create folder for each trial
        # create folder for each angle
        # create folder for each transducer count
        # create folder for each image
        # save image in the correct folder
        

        # next up setup beamformer variables and call beamformer
        img_res = [400, 600]
        image_x_range = [-0.019, 0.019]
        image_z_range = [0.005, 0.05]

        db_range = 50

        LATERAL_PIXEL_DENSITY_DEFAULT = 5

        img_config = ImageSettings( image_x_range[0],
                                    image_x_range[1],
                                    image_z_range[0],
                                    image_z_range[1],
                                    LATERAL_PIXEL_DENSITY_DEFAULT,
                                    data_loader_obj.transducer)

        channel_reduction = 128
        # active_elements = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
        active_elements = transducer_trials_values[i]
        angle_indices = angle_trials_values[j]
        print('angle_indices', angle_indices)     

        tx_string = 'PW_' + str(len(angle_indices)) + '_2'
        tx_strategy_indices = [tx_string, angle_indices]
        print(tx_strategy_indices)
        # angle_indices = list(range(75))

        print(data_loader_obj.transducer.set_active_elements(active_elements))
        print(data_loader_obj.transducer.active_elements)

        bf, rf_data = create_beamformer(tx_strategy_indices, img_config)
        print(rf_data.shape)

        trunkated_data = rf_data[:,active_elements,:]
        print(trunkated_data.shape)
        # beamformer 1
        print("Img Data for Beamformer " + angle_trial_name + " " + transducer_trial_name)
        img_data = bf.beamform(trunkated_data, numba_active=False)

        eval_obj = PicmusEval(img_data, bf)
        circle_pos = np.asarray([[-0.0105, 0.0281, 0.007, 0.004]])
        plot_name = "ultrasound_images/const_2/CNR_eval " + angle_trial_name + ", " + transducer_trial_name
        CNR_values = eval_obj.evaluate_circ_contrast(circle_pos,is_plot=True,
                                                     plot_name=plot_name)

        scatterer_pos = np.asarray([[-0.001, 0.018, 0.001, 0.001],
                                    [-0.001, 0.0277, 0.001, 0.001],
                                    [0.0001, 0.0375, 0.001, 0.001],
                                [-0.0104, 0.0375, 0.001, 0.001],
                                [0.0095, 0.0375, 0.001, 0.001]])
        plot_name = "ultrasound_images/const_2/FWHM_eval " + angle_trial_name + ", " + transducer_trial_name
        FWHM_x, FWHM_y = eval_obj.evaluate_FWHM(scatterer_pos,is_plot=True, 
                                                plot_name=plot_name)

        print("CNR values are [dB]:")
        print(str(CNR_values) + "\n")
        print("x-axis FWHM values are [m]:")
        print(str(FWHM_x))
        print("z-axis FWHM values are [m]:")
        print(str(FWHM_y))

        # create and save the image
        image_path = './ultrasound_images/const_2/'
        image_title = 'DAS ' + angle_trial_name + ", " + transducer_trial_name
        plot_image(np.abs(img_data), 
               scatters_coords_xz=None,
               elements_coords_xz=None,
               framework='matplotlib',
               title=image_title,
               image_x_range=image_x_range,
               image_z_range=image_z_range,
               db_range=db_range,
               colorscale='Greys',
               save_fig=True, 
               show=False,
               path_to_save=image_path)

        # Open the image database with write tags
        # 'w' overwrites the file
        # 'a' appends to the file
        with h5py.File('beamforming_image_database_const_2.hdf5', 'a') as f:
            # open angle trial data group
            if angle_trial_name not in f:
                angle_grp = f.create_group(angle_trial_name)
            else:
                angle_grp = f[angle_trial_name]
            # open transducer trial data group
            if transducer_trial_name not in angle_grp:
                trial_grp = angle_grp.create_group(transducer_trial_name)
            else:
                trial_grp = angle_grp[transducer_trial_name]

            print(transducer_trial_name)
            # save the values into the datasets
            if 'image' in trial_grp:
                del trial_grp['image']
            trial_grp.create_dataset('image', data=img_data)

            if 'CNR' in trial_grp:
                del trial_grp['CNR']
            trial_grp.create_dataset('CNR', data=CNR_values)

            if 'FWHM_x' in trial_grp:
                del trial_grp['FWHM_x']
            trial_grp.create_dataset('FWHM_x', data=FWHM_x)

            if 'FWHM_y' in trial_grp:
                del trial_grp['FWHM_y']
            trial_grp.create_dataset('FWHM_y', data=FWHM_y)

In [ ]:
# for loops
for j, angle_trial_name in enumerate(angle_trials_names):
    for i, transducer_trial_name in enumerate(transducer_trials_names):
        # create folder for each trial
        # create folder for each angle
        # create folder for each transducer count
        # create folder for each image
        # save image in the correct folder
        

        # next up setup beamformer variables and call beamformer
        img_res = [400, 600]
        image_x_range = [-0.019, 0.019]
        image_z_range = [0.005, 0.05]

        db_range = 50

        LATERAL_PIXEL_DENSITY_DEFAULT = 5

        img_config = ImageSettings( image_x_range[0],
                                    image_x_range[1],
                                    image_z_range[0],
                                    image_z_range[1],
                                    LATERAL_PIXEL_DENSITY_DEFAULT,
                                    data_loader_obj.transducer)

        channel_reduction = 128
        # active_elements = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
        active_elements = transducer_trials_values[i]
        angle_indices = angle_trials_values[j]
        print('angle_indices', angle_indices)

        #get maximum from angle indices and get the angle at that index
        max_angle_index = max(angle_indices)
        print(max_angle_index)
        max_angle = data_loader_obj.tx_strategy[1][max_angle_index]        

        tx_string = 'PW_' + str(len(angle_indices)) + '_' + '{:.3f}'.format(max_angle)
        tx_strategy_indices = [tx_string, angle_indices]
        print(tx_strategy_indices)
        # angle_indices = list(range(75))

        print(data_loader_obj.transducer.set_active_elements(active_elements))
        print(data_loader_obj.transducer.active_elements)

        bf, rf_data = create_beamformer(tx_strategy_indices, img_config)
        print(rf_data.shape)

        trunkated_data = rf_data[:,active_elements,:]
        print(trunkated_data.shape)
        # beamformer 1
        print("Img Data for Beamformer " + angle_trial_name + " " + transducer_trial_name)
        img_data = bf.beamform(trunkated_data, numba_active=False)

        eval_obj = PicmusEval(img_data, bf)
        circle_pos = np.asarray([[-0.0105, 0.0281, 0.007, 0.004]])
        plot_name = "ultrasound_images/var_deg/CNR_eval " + angle_trial_name + ", " + transducer_trial_name
        CNR_values = eval_obj.evaluate_circ_contrast(circle_pos,is_plot=True,
                                                     plot_name=plot_name)

        scatterer_pos = np.asarray([[-0.001, 0.018, 0.001, 0.001],
                                    [-0.001, 0.0277, 0.001, 0.001],
                                    [0.0001, 0.0375, 0.001, 0.001],
                                [-0.0104, 0.0375, 0.001, 0.001],
                                [0.0095, 0.0375, 0.001, 0.001]])
        plot_name = "ultrasound_images/var_deg/FWHM_eval " + angle_trial_name + ", " + transducer_trial_name
        FWHM_x, FWHM_y = eval_obj.evaluate_FWHM(scatterer_pos,is_plot=True, 
                                                plot_name=plot_name)

        print("CNR values are [dB]:")
        print(str(CNR_values) + "\n")
        print("x-axis FWHM values are [m]:")
        print(str(FWHM_x))
        print("z-axis FWHM values are [m]:")
        print(str(FWHM_y))

        # create and save the image
        image_path = './ultrasound_images/var_deg/'
        image_title = 'DAS ' + angle_trial_name + ", " + transducer_trial_name
        plot_image(np.abs(img_data), 
               scatters_coords_xz=None,
               elements_coords_xz=None,
               framework='matplotlib',
               title=image_title,
               image_x_range=image_x_range,
               image_z_range=image_z_range,
               db_range=db_range,
               colorscale='Greys',
               save_fig=True, 
               show=False,
               path_to_save=image_path)

        # Open the image database with write tags
        # 'w' overwrites the file
        # 'a' appends to the file
        with h5py.File('beamforming_image_database_var_ang.hdf5', 'a') as f:
            # open angle trial data group
            if angle_trial_name not in f:
                angle_grp = f.create_group(angle_trial_name)
            else:
                angle_grp = f[angle_trial_name]
            # open transducer trial data group
            if transducer_trial_name not in angle_grp:
                trial_grp = angle_grp.create_group(transducer_trial_name)
            else:
                trial_grp = angle_grp[transducer_trial_name]

            print(transducer_trial_name)
            # save the values into the datasets
            if 'image' in trial_grp:
                del trial_grp['image']
            trial_grp.create_dataset('image', data=img_data)

            if 'CNR' in trial_grp:
                del trial_grp['CNR']
            trial_grp.create_dataset('CNR', data=CNR_values)

            if 'FWHM_x' in trial_grp:
                del trial_grp['FWHM_x']
            trial_grp.create_dataset('FWHM_x', data=FWHM_x)

            if 'FWHM_y' in trial_grp:
                del trial_grp['FWHM_y']
            trial_grp.create_dataset('FWHM_y', data=FWHM_y)